This cell contains a couple preprocessing submethods which make up an overall preprocessing method. In addition, there is a supervised logistic regression method implemented with two visualizations. 

In [ ]:
##Imports

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA
import os
from functools import reduce
import random
import string
import uuid
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import PowerTransformer
from sklearn import svm

In [ ]:

def get_data(data):
    # Load the dataset
    df = pd.read_csv(data)

    # Convert the "Date" column to datetime
    df['Date'] = pd.to_datetime(df['GAME_DATE_EST'])

    #Filter the DataFrame for games between October 2011 and March 31, 2021
    start_date = '2012-10-01'
    end_date = '2021-03-31'
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    #Sort from first to last game to prevent data leakage
    sorted_df = filtered_df.sort_values(by='Date')
    
    

    
    return sorted_df
    
def get_totals(data): 
    #Load the dataset
    df = pd.read_csv(data)
    
    # Group by 'game_id' and calculate the median of total for each group
    medians = df.groupby('game_id')['total1'].median().reset_index()
    
    #Replace the df with just the median of the spread column
    merged_df = df.drop('total1', axis=1).drop_duplicates('game_id').merge(medians, on='game_id')
    
    #Keep only the game id and total in order to match this with other table
    reduced_df = df[['game_id', 'total1']]
    
    
    return reduced_df
    
def merge(games, totals): 
    df1 = get_data('games.csv')
    df2 = get_totals('nba_betting_totals.csv')
    #Strip columns of whitespace
    df1.columns = df1.columns.str.strip()
    df2.columns = df2.columns.str.strip()
    
    df1.columns = df1.columns.str.lower()  # Make all column names lowercase
    df2.columns = df2.columns.str.lower()
    
    #Merge on 'game_id'
    merged_df = pd.merge(df1, df2, on='game_id', how='inner')
    
   
    

    
    df_unique = merged_df.drop_duplicates(subset='game_id', keep='first')
    
    #Append label which is either 0 or 1 for binary classification. Make this the last row in data frame
    df_unique['hit_over'] = ((df_unique['pts_home'] + df_unique['pts_away']) > df_unique['total1']).astype(int)
    
    #Drop unnecessary columns for calculation
    df_unique.drop(['game_date_est', 'game_status_text', 'season', 'home_team_wins', 'team_id_home', 'team_id_away'], axis=1, inplace=True)
    df_unique.to_csv('datapreprocessed.csv')
    return df_unique


data = merge('games.csv', 'nba_betting_totals.csv')

#Get rolling stats for each team
def rolling_stats_home(teamid): 
    data = merge('games.csv', 'nba_betting_totals.csv')
    games_played = data[(data['home_team_id'] == teamid) | (data['visitor_team_id'] == teamid)]
    df = games_played.sort_values(by=['date'])
    
    # Creating a boolean mask where the value_of_interest is found in either ColumnA or ColumnB
    maskHome = (df['home_team_id'] == teamid)  
    maskAway = (df['visitor_team_id'] == teamid)

    # Using the mask to select rows and then choosing a specific set of columns for those rows
    # Adjust the selection of 'OtherColumn1_A' and 'OtherColumn2_A' based on your requirements
    selected_data_A = df.loc[maskHome, ['game_id', 'home_team_id', 'pts_home','fg_pct_home', 'ft_pct_home', 'fg3_pct_home', 'ast_home', 'reb_home', 'date']]
    
    selected_data_B = df.loc[maskAway, ['game_id', 'visitor_team_id', 'pts_away','fg_pct_away', 'ft_pct_away', 'fg3_pct_away', 'ast_away', 'reb_away', 'date']]
    
    #Generate unique id to identify which game the columns are attributed to
    random_id = uuid.uuid4()  # UUID object
    random_id_str = str(random_id)
    
    
    #Rename columns to prepare for concatenation
    selected_data_A.columns=['game_id', 'team_id', 'pts','fg_pct','ft_pct','fg3_pct','ast','reb','date']
    selected_data_B.columns=['game_id', 'team_id', 'pts','fg_pct','ft_pct','fg3_pct','ast','reb','date']
    
    #Concatenate the DataFrames
    df_stacked = pd.concat([selected_data_A, selected_data_B], ignore_index=True)
    df_sorted = df_stacked.sort_values(by='date', ascending=True) #This represents every nba game this team has played in the dataset. 
    
    
    ##Now we can compute the rolling averages and percent change for every team
    df_sorted.set_index('date', inplace=True)
    df_rolling_avg = df_sorted[['pts','fg_pct','ft_pct','fg3_pct','ast','reb']].rolling(window='7D', min_periods=1).mean().shift(1)
    
    df_rolling_avg.columns = ['avg_pts','avg_fg_pct','avg_ft_pct','avg_fg3_pct','avg_ast','avg_reb'] 
    
    df_concatenated = pd.concat([df_sorted, df_rolling_avg], axis=1)
    
    
   
    
    df_7_day_pct_change = df_sorted[['pts','fg_pct','ft_pct','fg3_pct','ast','reb']].pct_change(periods=7).shift(1)
    
    df_7_day_pct_change.columns = ['delta_pts','delta_fg_pct','delta_ft_pct','delta_fg3_pct','delta_ast','delta_reb']
    df_concatenated = pd.concat([df_sorted, df_rolling_avg, df_7_day_pct_change], axis=1)
    
    teamdf_clean = df_concatenated.drop(['pts','fg_pct','ft_pct','fg3_pct','ast','reb'], axis=1)
    mergedDf = pd.merge(teamdf_clean, data, on='game_id', how='inner')
    
    #Fill using backwards fill for simplicity
    mergedDf = mergedDf.fillna(method='bfill')
    mergedDf = mergedDf.drop('team_id', axis=1)
    droppedColumns = ['home_team_id', 'visitor_team_id', 'pts_home', 'fg_pct_home', 'ft_pct_home', 'fg3_pct_home', 'ast_home', 'reb_home', 'pts_away', 'fg_pct_away', 'ft_pct_away', 'fg3_pct_away', 'ast_away', 'reb_away', 'date', 'total1', 'hit_over']
    mergedDf = mergedDf.drop(droppedColumns, axis=1)
    newcolumns = ['game_id', 'avg_pts' + random_id_str, 'avg_fg_pct' + random_id_str, 'avg_ft_pct' + random_id_str, 'avg_fg3_pct' + random_id_str,'avg_ast' + random_id_str,'avg_reb' + random_id_str, 'delta_pts' + random_id_str,'delta_fg_pct' + random_id_str,'delta_ft_pct' + random_id_str,'delta_fg3_pct' + random_id_str,'delta_ast' + random_id_str,'delta_reb' + random_id_str]
    mergedDf.columns = newcolumns
    mergedDf.to_csv(str(teamid) + 'Data.csv', index=False)

    return mergedDf
    



#df.to_csv('datatrial1.csv', index=False)

#print(rolling_stats(1610612737, '2012-11-02'))

In [ ]:
##Now for training and testing with Logistic Regression
##Split the test 
data = rolling_stats_home(1610612739)

n_test = 60

train_df = data[20:-n_test]  # Training set: all rows except the last 'n_test' rows
test_df = data[-n_test:]   # Test set: the last 'n_test' rows


unneeded_columns = ['game_id', 'hit_over', 'total1', 'date']
# Scaling all the variables to a range of 0 to 1

X_train = train_df.drop(columns=unneeded_columns, axis=1)
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train)
x = pd.DataFrame(scaler.transform(X_train))
features = x.columns.values  
    
y_train = train_df['hit_over']
X_test = test_df.drop(columns=unneeded_columns, axis=1)
y_test = test_df['hit_over']   



# Create an instance of the model
logreg = LogisticRegression()

# Fit the model with the training data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)



# Evaluate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))





# Calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
roc_auc = auc(fpr, tpr)

# Plotting the ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


pca = PCA(n_components=2)
X_r = pca.fit_transform(X_train)

# Fit the logistic regression model on the reduced data for visualization
logreg.fit(X_r, y_train)

# Plotting the reduced data
plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1], ['Class 0', 'Class 1']):
    plt.scatter(X_r[y_train == i, 0], X_r[y_train == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of dataset')
plt.show()

The following cell will contain another preprocessing method. In addition, we will implement an SVM classifier.

In [ ]:
rolling_stats_home(1610612739)

In [ ]:
data = 'cleandata.csv'

In [4]:

#This creates the rolling stats for every team for the 10/1/2012 - 3/31/21
#It is a collection of every game recorded and the prior 7 day average of their statistics

data = pd.read_csv('datapreprocessed.csv')
print(data['home_team_id'].unique())

def createData(): 
    for team in data['home_team_id'].unique(): 
        rolling_stats_home(team)
        
createData()


[1610612759 1610612766 1610612753 1610612737 1610612739 1610612752
 1610612738 1610612760 1610612750 1610612756 1610612747 1610612744
 1610612740 1610612764 1610612754 1610612751 1610612748 1610612741
 1610612742 1610612749 1610612746 1610612745 1610612761 1610612763
 1610612755 1610612758 1610612743 1610612762 1610612757 1610612765]


/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['hit_over'] = ((df_unique['pts_home'] + df_unique['pts_away']) > df_unique['total1']).astype(int)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(['game_date_est', 'game_status_text', 'season', 'home_team_wins', 'team_id_home', 'team_id_away'], axis=1, inplace=True)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/25

/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['hit_over'] = ((df_unique['pts_home'] + df_unique['pts_away']) > df_unique['total1']).astype(int)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(['game_date_est', 'game_status_text', 'season', 'home_team_wins', 'team_id_home', 'team_id_away'], axis=1, inplace=True)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/25

/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['hit_over'] = ((df_unique['pts_home'] + df_unique['pts_away']) > df_unique['total1']).astype(int)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(['game_date_est', 'game_status_text', 'season', 'home_team_wins', 'team_id_home', 'team_id_away'], axis=1, inplace=True)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/25

/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['hit_over'] = ((df_unique['pts_home'] + df_unique['pts_away']) > df_unique['total1']).astype(int)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(['game_date_est', 'game_status_text', 'season', 'home_team_wins', 'team_id_home', 'team_id_away'], axis=1, inplace=True)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/25

/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['hit_over'] = ((df_unique['pts_home'] + df_unique['pts_away']) > df_unique['total1']).astype(int)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/2531063174.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(['game_date_est', 'game_status_text', 'season', 'home_team_wins', 'team_id_home', 'team_id_away'], axis=1, inplace=True)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/25

In [5]:
directory = '/Users/amanpuri/Desktop/Spring 2024/ML 4641/Project/uniquecolumns2'
dataframes = []
def combineData(directory): 
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):  # Filter for CSV files
        # Create the full path to the file
            file_path = os.path.join(directory, filename)
            
        # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
        # Display the DataFrame (optional, for verification in Jupyter Notebook)
            dataframes.append(df)
    big_dataframe = reduce(lambda left, right: pd.merge(left, right, on='game_id', how='outer'), dataframes)
    big_dataframe.to_csv('bigdataframetest.csv')
    return big_dataframe
combineData(directory)

,game_id,avg_ptsa3402417-df61-424b-ab01-ac366f190be7,avg_fg_pcta3402417-df61-424b-ab01-ac366f190be7,avg_ft_pcta3402417-df61-424b-ab01-ac366f190be7,avg_fg3_pcta3402417-df61-424b-ab01-ac366f190be7,avg_asta3402417-df61-424b-ab01-ac366f190be7,avg_reba3402417-df61-424b-ab01-ac366f190be7,delta_ptsa3402417-df61-424b-ab01-ac366f190be7,delta_fg_pcta3402417-df61-424b-ab01-ac366f190be7,delta_ft_pcta3402417-df61-424b-ab01-ac366f190be7,...,avg_ft_pct19979801-8ee7-43c7-8ef7-54b293179e50,avg_fg3_pct19979801-8ee7-43c7-8ef7-54b293179e50,avg_ast19979801-8ee7-43c7-8ef7-54b293179e50,avg_reb19979801-8ee7-43c7-8ef7-54b293179e50,delta_pts19979801-8ee7-43c7-8ef7-54b293179e50,delta_fg_pct19979801-8ee7-43c7-8ef7-54b293179e50,delta_ft_pct19979801-8ee7-43c7-8ef7-54b293179e50,delta_fg3_pct19979801-8ee7-43c7-8ef7-54b293179e50,delta_ast19979801-8ee7-43c7-8ef7-54b293179e50,delta_reb19979801-8ee7-43c7-8ef7-54b293179e50
0,11700001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11700002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21200013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21200014,84.0,0.377,0.826,0.412,18.0,48.0,0.095238,-0.018568,0.066586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21200015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7632,41700317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7633,41700401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7634,41700402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7635,41700403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
fullDF = pd.read_csv('bigdataframetest.csv')


##Need to shift all values as far left as possible
# Function to shift values left
def shift_left(row):
    # Drop all NaNs and convert the row to a list
    filtered = row.dropna().tolist()
    # Calculate the number of NaNs to append at the end of the row
    nans = [np.nan] * (len(row) - len(filtered))
    # Combine non-NaN values with NaNs at the end
    return pd.Series(filtered + nans)
# Apply the function along axis=1 to affect rows
df_cleaned = fullDF.apply(shift_left, axis=1)
df_cleaned[1] = df_cleaned[1].astype(int)
df_cleaned = df_cleaned.drop(0, axis=1)
columns_to_drop = list(range(26, 362))
df_cleaned = df_cleaned.drop(columns=columns_to_drop)
##Rename Columns and export
columnlabels = ['game_id','avg_pts_1','avg_fg_pct_1','avg_ft_pct_1','avg_fg3_pct_1','avg_ast_1','avg_reb_1','delta_pts_1','delta_fg_pct_1','delta_ft_pct_1','delta_fg3_pct_1','delta_ast_1','delta_reb_1', 'avg_pts_2','avg_fg_pct_2','avg_ft_pct_2','avg_fg3_pct_2','avg_ast_2','avg_reb_2','delta_pts_2','delta_fg_pct_2','delta_ft_pct_2','delta_fg3_pct_2','delta_ast_2','delta_reb_2']
df_cleaned.columns = columnlabels
df_cleaned.to_csv('bigdataframetestshifted.csv', index=False)

In [7]:
##Combine them into ultimate preprocessed and export 
allGameStats = pd.read_csv('datapreprocessed.csv')
allTeamStats = pd.read_csv('bigdataframetestshifted.csv')

# Perform an inner join on the 'Key' column
result_inner = pd.merge(allGameStats, allTeamStats, on='game_id', how='inner')
result_inner = result_inner.drop(result_inner.columns[0], axis=1)
columns_to_drop = ['pts_home', 'fg_pct_home', 'ft_pct_home', 'fg3_pct_home', 'ast_home', 'reb_home','pts_away', 'fg_pct_away', 'ft_pct_away', 'fg3_pct_away', 'ast_away', 'reb_away']
result_inner = result_inner.drop(columns=columns_to_drop)
result_inner.to_csv('finalpreprocessed.csv', index=False)




In [ ]:
data_types = fullDF.dtypes
print(data_types)

In [8]:
##Now to implement SVM on the preprocessed dataset

##Prepare data into feature matrix and label matrix
df = pd.read_csv('finalpreprocessed.csv')


df.fillna(method='bfill', inplace=True)

# Replace infinite values only in numeric columns
for column in df.select_dtypes(include=[np.number]).columns:
    if np.isinf(df[column]).any():
        df[column].replace([np.inf, -np.inf], 0, inplace=True) 
    
X = df.drop(['game_id', 'home_team_id', 'visitor_team_id', 'date', 'hit_over'], axis=1)
y = df['hit_over']


# Select only columns where all values are greater than 1
columns_to_scale = [col for col in X.columns if df[col].min() > 1]

# Initialize the scaler
scaler = StandardScaler()

# Scale selected columns and update the DataFrame
X_scaled = scaler.fit_transform(X[columns_to_scale])
X[columns_to_scale] = X_scaled
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




# Creating the SVM model with a linear kernel
model = svm.SVC(kernel='linear', gamma=10, C=1)

# Training the SVM model
model.fit(X_train, y_train)

# Predicting the Test set results
y_pred = model.predict(X_test)

# Evaluating the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))




/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/3739588700.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
/var/folders/2b/dmr7rjb502b8l2wyn77brfnc0000gn/T/ipykernel_46445/3739588700.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].replace([np.inf, -np.inf], 0, inplace=True)


Accuracy: 0.4993455497382199

Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.62      0.56       776
           1       0.49      0.38      0.43       752

    accuracy                           0.50      1528
   macro avg       0.50      0.50      0.49      1528
weighted avg       0.50      0.50      0.49      1528

